In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sparse
from sklearn.cluster import KMeans
from itertools import permutations
from copy import deepcopy

plt.style.use("ggplot")

def ScBM_generator(row_label, column_label, link_matrices, density=1):
    """Returns a multi-layer stochastic co-blockmodel graph (L*Ky*Kz). see also nx.stochastic_block_model"""
    p_matrices = link_matrices[:,row_label][:,:,column_label]
    p_matrices = p_matrices - np.einsum('jk,ik->jki', np.diagonal(p_matrices, axis1=1, axis2=2), np.eye(p_matrices.shape[2])) 
    return np.random.binomial(1, density*p_matrices)


def k_eigenvectors(scbm, K, method, row=True):
    """ Returns the K leading eigenvectors;  method: {"sum", "qsum"(include qsum_debias)} """
    # the sum of adjacency matrices
    if method == "sum":
        u, _, v = np.linalg.svd(np.sum(scbm, axis=0))
        vs = u[:, 0:K] if row else v[0:K, ].T  # K leading eigenvectors
        return vs

    if method == "qsum": 
        if row:
            square = np.zeros_like(scbm)
            for l in range(scbm.shape[0]):
                Al_sparse = sparse.coo_matrix(scbm[l])
                square[l] = Al_sparse.dot(Al_sparse.T).toarray()

            # the sum of squared adjacency matriecs
            _, v = np.linalg.eigh(np.sum(square, axis=0))
            vs = v[:, ::-1][:, 0:K]  # ascending order
            # the bias-adjusted sum of squared adjacency matriecs
            _, v = np.linalg.eigh(np.sum(square, axis=0) - np.diag(np.sum(scbm, axis=0).sum(axis=1)))
            vs_debias = v[:, ::-1][:, 0:K]

        else:
            square = np.zeros_like(scbm)
            for l in range(scbm.shape[0]):
                Al_sparse = sparse.coo_matrix(scbm[l])
                square[l] = Al_sparse.T.dot(Al_sparse).toarray()

            _, u = np.linalg.eigh(np.sum(square, axis=0))
            vs = u[:, ::-1][:, 0:K]
            _, u = np.linalg.eigh(np.sum(square, axis=0) - np.diag(np.sum(scbm, axis=0).sum(axis=0)))
            vs_debias = u[:, ::-1][:, 0:K]
        return vs, vs_debias 


def block_sums(scbm, clusters_est, K_clusters):
    """ block sums B """
    B = np.zeros((scbm.shape[0], scbm.shape[1], K_clusters))
    for k in range(K_clusters):
        mask = (clusters_est == k).reshape(1, 1, -1)
        B[:, :, k] = (scbm * mask).sum(axis=2)
    return B

def initial_param(scbm, K_clusters, clusters_init):
    masks = [(clusters_init == k).reshape(1, -1, 1) for k in range(K_clusters)]
    obs = np.zeros((scbm.shape[0], K_clusters, K_clusters))
    for i in range(K_clusters):
        for j in range(K_clusters):
            obs[:, i, j] = np.sum(scbm * masks[i] * masks[j].transpose(0, 2, 1), axis=(1, 2))
    P_hat = obs / np.outer(np.bincount(clusters_init, minlength=K_clusters), np.bincount(clusters_init, minlength=K_clusters))

    pi = np.bincount(clusters_init, minlength=K_clusters) / scbm.shape[1]
    R_hat = np.diag(pi)
    Lambda_hat = scbm.shape[1] * R_hat @ P_hat
    return pi, P_hat, Lambda_hat.transpose((0, 2, 1))


def PL(scbm, K_clusters, clusters_init, T=20, maxiter=1000, tol=1e-4, row=True):
    if not row: scbm = scbm.transpose((0, 2, 1))
    pi_hat, _, lambda_hat = initial_param(scbm, K_clusters, clusters_init)
    r_est = clusters_init
    for t in range(T):
        B = block_sums(scbm, r_est, K_clusters)
        inner_iter = 0
        while inner_iter < maxiter:
            # eq 9
            lambda_hat = np.where(lambda_hat<=0, 1e-5, lambda_hat) # avoid log with a negative input
            temp = B[:, :, np.newaxis, :] * np.log(lambda_hat[:, np.newaxis, :, :]) - lambda_hat[:, np.newaxis, :, :]
            temp = np.sum(temp, axis=(0, 3)) - np.max(np.sum(temp, axis=(0, 3)), axis=1)[:, np.newaxis] # avoid overflow
            temp = np.exp(temp) * pi_hat

            # temp = np.exp(np.sum(temp, axis=(0, 3))) * pi_hat
            pi_mat = temp / temp.sum(axis=1)[:, np.newaxis]
            pi_hat_new = np.mean(pi_mat, axis=0)
            temp = np.sum(pi_mat, axis=0)[:, np.newaxis]
            temp = np.where(temp <= 0, 1e-5, temp)
            lambda_hat_new = np.sum(pi_mat[np.newaxis, :, :, np.newaxis] * B[:, :, np.newaxis, :], axis=1) / temp
            error_pi, error_lambda = np.linalg.norm(pi_hat_new - pi_hat), np.linalg.norm(lambda_hat_new - lambda_hat)
            pi_hat, lambda_hat = pi_hat_new, lambda_hat_new
            if max(error_pi, error_lambda) < tol:
                break
            inner_iter += 1
        
        # eq 12
        r_est = np.argmax(pi_mat, axis=1)
    return r_est

def PPL(scbm, K_clusters, clusters_init, T=20, maxiter=200, tol=1e-3, row=True):
    if row: scbm = scbm.transpose((0, 2, 1))
    pi_hat, P_hat, _ = initial_param(scbm, K_clusters, clusters_init)
    clusters_est = clusters_init
    for t in range(T):
        inner_iter = 0
        while inner_iter < maxiter:
            # E-step
            P_hat = np.where(P_hat<=0, 1e-5, P_hat)
            P_hat = np.where(P_hat>=1, 1 - 1e-5, P_hat)
            P_temp = P_hat[:, :, clusters_est]
            temp = scbm[:, :, np.newaxis, :] * np.log(P_temp[:, np.newaxis, :, :]) + (1 - scbm[:, :, np.newaxis, :]) * np.log(1 - P_temp[:, np.newaxis, :, :])
            pi_hat = np.where(pi_hat<=0, 1e-5, pi_hat)
            temp = np.exp(np.sum(temp, axis=(0, 3)) + np.log(pi_hat) - np.max(np.sum(temp, axis=(0, 3)) + np.log(pi_hat), axis=1)[:, np.newaxis])
            tau_mat = temp / temp.sum(axis=1)[:, np.newaxis]

            # M-step
            pi_hat_new = np.mean(tau_mat, axis=0)
            indicator = np.zeros((scbm.shape[1], K_clusters))
            indicator[np.arange(scbm.shape[1]), clusters_est] = 1
            temp_num = scbm[:, :, :, np.newaxis, np.newaxis] * tau_mat[np.newaxis, :, np.newaxis, :, np.newaxis] * indicator[np.newaxis, np.newaxis, :, np.newaxis, :]
            temp_num = np.sum(temp_num, axis=(1, 2)) 
            temp_denom = tau_mat[:, np.newaxis, :, np.newaxis] * indicator[np.newaxis, :, np.newaxis, :]
            temp_denom = np.sum(temp_denom, axis=(0, 1))
            temp_denom = np.where(temp_denom <= 0, 1e-5, temp_denom)
            P_hat_new = temp_num / temp_denom
            error_pi, error_P = np.linalg.norm(pi_hat_new - pi_hat), np.linalg.norm(P_hat_new - P_hat)
            pi_hat, P_hat = pi_hat_new, P_hat_new
            if max(error_pi, error_P) < tol:
                break
            inner_iter += 1
        
        # eq 2.4
        P_hat = np.where(P_hat<=0, 1e-5, P_hat)
        P_hat = np.where(P_hat>=1, 1 - 1e-5, P_hat)
        P_hat = P_hat.transpose((0, 2, 1))
        temp = tau_mat[np.newaxis, :, np.newaxis, np.newaxis, :] * (scbm[:, :, :, np.newaxis, np.newaxis] * np.log(P_hat[:, np.newaxis, np.newaxis, :, :]) + (1 - scbm[:, :, :, np.newaxis, np.newaxis]) * np.log(1 - P_hat[:, np.newaxis, np.newaxis, :, :]))
        clusters_est_new = np.argmax(np.sum(temp, axis=(0, 1, 4)), axis=1)
        error_cluster = np.count_nonzero(clusters_est_new - clusters_est)
        clusters_est = clusters_est_new
        if  error_cluster <= 0:
            break

    return clusters_est


def min_mis_error(true_label, kmeans_labels, K_clusters):
    """optimal permutation"""
    kmeans_labels_p = deepcopy(kmeans_labels)
    location = []
    for i in range(K_clusters):
        location.append(np.where(kmeans_labels_p == i))
    
    error_min = 1
    for cp in permutations(range(K_clusters), K_clusters):
        for j, r in enumerate(cp):
            kmeans_labels_p[location[j]] = r
        error = np.sum(kmeans_labels_p != true_label) / true_label.shape[0]
        if error <= error_min:
            error_min = error
            best_p = cp
    
    for j, r in enumerate(best_p):
        kmeans_labels_p[location[j]] = r
        
    return error_min    

## Y = Z

### The row and column membership matrices are identical

In [ ]:
def repeat_n_couple(B_1, B_2, layer, n_nodes, K, K_sum, K_clusters, density, n_trials=100, row=True, initial="random"):
    n_nodes = np.atleast_1d(n_nodes)
    mis_errors = np.zeros((3, n_trials, n_nodes.shape[0])) #methods = ["PL", "PPL", "dsog"]
    
    for j in range(n_nodes.shape[0]):
        size_communities = np.array([[0.33, 0.33, 0.34], [0.33, 0.33, 0.34]]) * n_nodes[j]
        size_communities = size_communities.astype(int)
        num_communities = [3, 3]
        true_row_label = np.random.choice(np.repeat(range(num_communities[0]), size_communities[0]), replace=False, size=n_nodes[j])
        true_column_label = true_row_label
        link_matrices = np.concatenate((np.tile(B_1, (int(layer/2), 1, 1)), np.tile(B_2, (int(layer/2) + layer % 2, 1, 1))))

        if row:
            true_label = true_row_label
        else:
            true_label = true_column_label

        k_means = KMeans(init="k-means++", n_clusters=K_clusters, n_init=n_nodes[j])

        # initial value 
        scbm = ScBM_generator(true_row_label, true_column_label, link_matrices, density)
        if initial == "dsog":
            _, vs = k_eigenvectors(scbm, K, method="qsum", row=row) # DSoG
            k_means.fit(vs)
            initial_value = k_means.labels_
        elif initial == "sum":
            vs = k_eigenvectors(scbm, K_sum, method="sum", row=row) # Sum
            k_means.fit(vs)
            initial_value = k_means.labels_
        else:
            initial_value = np.random.choice(K_clusters, n_nodes[j])


        for i in range(n_trials):
            scbm = ScBM_generator(true_row_label, true_column_label, link_matrices, density)
            print("#Nodes: %s,  Trial: %s;    " %(n_nodes[j], i+1))
            _, vs_debias = k_eigenvectors(scbm, K, method="qsum", row=row)
            k_means.fit(vs_debias)
            mis_errors[2, i, j] =min_mis_error(true_label, k_means.labels_, k_means.n_clusters)

            pl_label = PL(scbm, K_clusters, initial_value, row=row)  
            mis_errors[0, i, j] = min_mis_error(true_label, pl_label, K_clusters)

            ppl_label = PPL(scbm, K_clusters, initial_value, row=row) 
            mis_errors[1, i, j] = min_mis_error(true_label, ppl_label, K_clusters)

    return mis_errors

In [ ]:
U = np.array([[1/2, 1/2, -np.sqrt(2)/2], [1/2, 1/2, np.sqrt(2)/2], [np.sqrt(2)/2, -np.sqrt(2)/2, 0]])
V = np.array([[np.sqrt(2)/2, -np.sqrt(2)/2, 0], [1/2, 1/2, -np.sqrt(2)/2], [1/2, 1/2, np.sqrt(2)/2]])
Lambda1 = np.diag([1.5, 0.2, 0.4])
Lambda2 = np.diag([1.5, 0.2, -0.4])
B_1 = U@Lambda1@V.T
B_2 = U@Lambda2@V.T

K_clusters = 3
K = 3 # K leading eigenvectors
K_sum = 2

n_nodes = np.arange(300, 1501, 100)
# # random initial value
# # row community reconstruction
# np.random.seed(2024)
# row_10l_couple_random = repeat_n_couple(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="random")
# # column community reconstruction
# np.random.seed(2025)
# column_10l_couple_random = repeat_n_couple(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="random", row=False)

# # initial value Sum
# # row community reconstruction
# np.random.seed(2024)
# row_10l_couple_sum = repeat_n_couple(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="sum")
# # column community reconstruction
# np.random.seed(2025)
# column_10l_couple_sum = repeat_n_couple(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="sum", row=False)

# initial value DSoG
# row community reconstruction
np.random.seed(2024)
row_10l_couple_dsog = repeat_n_couple(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="dsog")
# column community reconstruction
np.random.seed(2025)
column_10l_couple_dsog = repeat_n_couple(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="dsog", row=False)

In [ ]:
sns.set_theme(style="ticks")
font = {"family": "Times New Roman", "weight": "normal", "size": 13}    
color_list = ["#534439", "#9A5F30", "#62B17C"]

def plot_mis(mis_errors, methods_list=["ML-PL", "ML-PPL", "DSoG"]):
    """mis_errors is a (n_methodes, n_trials, n_densities) ndarray"""
    plt.figure(figsize=(6,3.6))
    sns.lineplot(pd.DataFrame(mis_errors.mean(axis=1).T, n_nodes, methods_list), 
                 markers=["o", "X", "p"], dashes=False, palette=color_list, linewidth=2)
    plt.xlabel("The number of nodes n", font)
    plt.ylabel("Misclassification rate", font)
    plt.legend(ncol=3, loc=1, prop=font)
    plt.ylim(-0.015, 0.38)
    plt.tight_layout()
    plt.show()

# plot_mis(row_10l_couple_random)
# plot_mis(column_10l_couple_random)
# plot_mis(row_10l_couple_sum)
# plot_mis(column_10l_couple_sum)
plot_mis(row_10l_couple_dsog)
plot_mis(column_10l_couple_dsog)

## Y != Z

### The row and column membership matrices are different

In [ ]:
def repeat_n(B_1, B_2, layer, n_nodes, K, K_sum, K_clusters, density, n_trials=100, row=True, initial="random"):
    n_nodes = np.atleast_1d(n_nodes)
    mis_errors = np.zeros((3, n_trials, n_nodes.shape[0])) #methods = ["PL", "PPL", "dsog"]
    
    for j in range(n_nodes.shape[0]):
        size_communities = np.array([[0.33, 0.33, 0.34], [0.34, 0.33, 0.33]]) * n_nodes[j]
        size_communities = size_communities.astype(int)
        num_communities = [3, 3]
        true_row_label = np.random.choice(np.repeat(range(num_communities[0]), size_communities[0]), replace=False, size=n_nodes[j])
        true_column_label = np.random.choice(np.repeat(range(num_communities[1]), size_communities[1]), replace=False, size=n_nodes[j])
        link_matrices = np.concatenate((np.tile(B_1, (int(layer/2), 1, 1)), np.tile(B_2, (int(layer/2) + layer % 2, 1, 1))))

        if row:
            true_label = true_row_label
        else:
            true_label = true_column_label

        k_means = KMeans(init="k-means++", n_clusters=K_clusters, n_init=n_nodes[j])
        
        # initial value 
        scbm = ScBM_generator(true_row_label, true_column_label, link_matrices, density)
        if initial == "dsog":
            _, vs = k_eigenvectors(scbm, K, method="qsum", row=row) # DSoG
        else:
            vs = k_eigenvectors(scbm, K_sum, method="sum", row=row) # Sum
        k_means.fit(vs)
        initial_value = k_means.labels_


        for i in range(n_trials):
            scbm = ScBM_generator(true_row_label, true_column_label, link_matrices, density)
            print("#Nodes: %s,  Trial: %s;    " %(n_nodes[j], i+1))
            _, vs_debias = k_eigenvectors(scbm, K, method="qsum", row=row)
            k_means.fit(vs_debias)
            mis_errors[2, i, j] =min_mis_error(true_label, k_means.labels_, k_means.n_clusters)

            pl_label = PL(scbm, K_clusters, initial_value, row=row)
            mis_errors[0, i, j] = min_mis_error(true_label, pl_label, K_clusters)

            ppl_label = PPL(scbm, K_clusters, initial_value, row=row)
            mis_errors[1, i, j] = min_mis_error(true_label, ppl_label, K_clusters)

    return mis_errors


In [ ]:
U = np.array([[1/2, 1/2, -np.sqrt(2)/2], [1/2, 1/2, np.sqrt(2)/2], [np.sqrt(2)/2, -np.sqrt(2)/2, 0]])
V = np.array([[np.sqrt(2)/2, -np.sqrt(2)/2, 0], [1/2, 1/2, -np.sqrt(2)/2], [1/2, 1/2, np.sqrt(2)/2]])
Lambda1 = np.diag([1.5, 0.2, 0.4])
Lambda2 = np.diag([1.5, 0.2, -0.4])
B_1 = U@Lambda1@V.T
B_2 = U@Lambda2@V.T
K_clusters = 3
K = 3 # K leading eigenvectors
K_sum = 2

n_nodes = np.arange(300, 1501, 100)
# # random initial value
# # row community reconstruction
# np.random.seed(2024)
# row_10l_random = repeat_n(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="random")
# # column community reconstruction
# np.random.seed(2025)
# column_10l_random = repeat_n(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="random", row=False)

# # initial value Sum
# # row community reconstruction
# np.random.seed(2024)
# row_10l_sum = repeat_n(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="sum")
# # column community reconstruction
# np.random.seed(2025)
# column_10l_sum = repeat_n(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="sum", row=False)

# initial value DSoG
# row community reconstruction
np.random.seed(2024)
row_10l_dsog = repeat_n(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="dsog")
# column community reconstruction
np.random.seed(2025)
column_10l_dsog = repeat_n(B_1, B_2, 10, n_nodes, K, K_sum, K_clusters, 0.1, n_trials=50, initial="dsog", row=False)


In [ ]:
sns.set_theme(style="ticks")
font = {"family": "Times New Roman", "weight": "normal", "size": 13}    
color_list = ["#534439", "#9A5F30", "#62B17C"]

def plot_mis(mis_errors, methods_list=["ML-PL", "ML-PPL", "DSoG"]):
    """mis_errors is a (n_methodes, n_trials, n_densities) ndarray"""
    plt.figure(figsize=(6,3.6))
    sns.lineplot(pd.DataFrame(mis_errors.mean(axis=1).T, n_nodes, methods_list), 
                 markers=["o", "X", "p"], dashes=False, palette=color_list, linewidth=2)
    plt.xlabel("The number of nodes n", font)
    plt.ylabel("Misclassification rate", font)
    plt.legend(ncol=3, loc=1, prop=font)
    plt.ylim(-0.015, 0.42)
    plt.tight_layout()
    plt.show()

# plot_mis(row_10l_random)
# plot_mis(column_10l_random)
# plot_mis(row_10l_sum)
# plot_mis(column_10l_sum)
plot_mis(row_10l_dsog)
plot_mis(column_10l_dsog)